# Limpel-Ziv-Welch Decoder
This python note book contains code to decode a binary LZW encoded message into ASCII characters, the initial dictionary used can be found [here](https://theasciicode.com.ar/)

In [1]:
import copy
import pandas as pd

## Dictionary Creation

In [2]:
def front_fill_zeros(binary_number, desired_length):
    difference = desired_length - len(binary_number)
    # print(f"Difference {difference}")
    if difference == 0: return binary_number

    zeros = difference * '0'
    return zeros + binary_number

df = pd.read_csv('ascii-table.csv')
df = df.drop(columns=['Number'])

binary_numbers = []
for i in range(128):
    entry = bin(i)[2:]
    binary_numbers.append(entry)

max_length = max(len(number) for number in binary_numbers)
binary_numbers_new = [front_fill_zeros(i, max_length) for i in binary_numbers]
df['Binary'] = binary_numbers_new
df.at[0, 'symbol'] = 'NULL'

print(df)

    symbol                                        Description   Binary
0     NULL                                   (Null character)  0000000
1      SOH                                  (Start of Header)  0000001
2      STX                                    (Start of Text)  0000010
3      ETX                                      (End of Text)  0000011
4      EOT                              (End of Transmission)  0000100
..     ...                                                ...      ...
123      {                         (curly brackets or braces)  1111011
124      |  (vertical-bar, vbar, vertical line or vertical...  1111100
125      }                         (curly brackets or braces)  1111101
126      ~                               (Tilde ; swung dash)  1111110
127    DEL                                           (Delete)  1111111

[128 rows x 3 columns]


## Decoding fucntions

In [3]:
def get_symbol_length(dictionary: pd.DataFrame) -> int:
    return len(dictionary['Binary'].iloc[0])


def get_symbol(binary_code: str, dictionary:pd.DataFrame) -> str:
    row = dictionary.loc[dictionary['Binary'] == binary_code]
    return row['symbol'].values[0]


def update_num_bits(dictionary: pd.DataFrame) -> bool:
    last_entry = dictionary['Binary'].iloc[-1]
    # print(f"Last entry {last_entry}")
    if '0' in last_entry:
        return False
    return True


def add_to_dictionary(entry: str, dictionary: pd.DataFrame) -> pd.DataFrame:
    binary_num = bin(dictionary.shape[0])[2:] # [2:] to remove the '0b' from the start of the binary number
    previous_binary_num = dictionary['Binary'].iloc[-1]

    # print(f"New binary number {dictionary.shape[0]}")

    if len(binary_num) != len(previous_binary_num):
        # print(f"Transitioning from {len(previous_binary_num)} bits to {len(binary_num)} bits")
        dictionary = back_fill_zeros_dictionary(dictionary)

    new_row = pd.Series([entry, "-", binary_num], index=dictionary.columns).T
    new_row_df = pd.DataFrame(new_row).T
    dictionary = pd.concat([dictionary, new_row_df], ignore_index=True) # type: ignore
    return dictionary


def back_fill_zeros_dictionary(dictionary: pd.DataFrame) -> pd.DataFrame:
    binaries = dictionary['Binary'].values
    new_length = len(binaries[0]) + 1
    # print(f"new length {new_length}")
    new_binaries = [front_fill_zeros(i, new_length) for i in binaries]
    dictionary['Binary'] = new_binaries
    # print(dictionary)
    return dictionary


def update_last_entry(symbol: str, dictionary: pd.DataFrame) -> pd.DataFrame:
    last_entry_symbol = dictionary['symbol'].iloc[-1]
    print(f"updating {last_entry_symbol} to { last_entry_symbol + symbol[0]}")
    dictionary.at[dictionary.shape[0] - 1, 'symbol'] = last_entry_symbol + symbol[0]
    return dictionary


def decode(binary_message: str, dictionary: pd.DataFrame) -> str:
    binary_message_copy = copy.copy(binary_message)

    ascii_message = ""
    first = True
    while binary_message_copy != "":

        symbol_length = get_symbol_length(dictionary)
        # print(f"Symbol length: {symbol_length}")
        symbol_code = binary_message_copy[:symbol_length]
        binary_message_copy = binary_message_copy.replace(symbol_code, "", 1)

        symbol = get_symbol(symbol_code, dictionary)
        # print(f"Symbol from dictionary: {symbol}")
        ascii_message = ascii_message + symbol


        if update_num_bits(dictionary):
            # print("Updating bits in dictionary")
            dictionary = back_fill_zeros_dictionary(dictionary)

        if not first:
            # print(f"updating dictionary")
            # print(dictionary)
            dictionary = update_last_entry(symbol, dictionary)
            # print(dictionary)
        
        dictionary = add_to_dictionary(symbol, dictionary)

        first = False

    return ascii_message


## Decoding usage

In [4]:

# binary_message = '11111110110000101100010000000000110010010000001011000100110001100000011'
# binary_message = '110000100001010010000110110111101101101011011010111010101101110011010010110001101100001011101000110100101101111011011100010000001110011011110010111001101110100011001010110110100100000011001000110010101110011011010010110011110001110011100100110010101110001011101010110100110011101011100110010000001110100011010000110010110010110100110001001101010001110011011110110011000100000011010000110000101110010011001000111011110110000101001110110000101101110011001001000111110101100011101001011001110011101100011111001000110010011011011010111001100101100000010100111011101101001101001011010010010100110001000000110111101100010011010100110010101100011'
# message = decode(binary_message, df)

lzw_file_path = 'lzw_message.txt'
with open(lzw_file_path) as f:
    message = f.read()

print(message)
# message = 'asd  asd s'
ascii_message = decode(message, df)
ascii_message = ascii_message.replace("LF", "\n")
ascii_message = ascii_message.replace('SQO', '[')
ascii_message = ascii_message.replace('SQC', ']')
ascii_message = ascii_message.replace('RBO', '(')
ascii_message = ascii_message.replace('RBC', ')')
print(f"\nFinal message: {ascii_message}")
ascii_file_path = 'ascii_message.txt'
ascii_message = ascii_message.encode('utf-8')

with open(ascii_file_path, 'wb') as f:
    f.write(ascii_message)

1000011011011110110110101101101011101010110111001101001011000110110000101110100011010010110111101101110001000000111001101111001011100110111010001100101011011010010000001100100011001010111001101101001011001111000110001110010011001010111000101110101011010011001101101110011001000000111010001101000011001011001010010010110100110001000110001101111011001100010000001101000011000010111001001100100011101111010111010100101011000010110111001100100100011011010101001110100101100011001101110001101100011111001000101101101011100110010110000001010011101110110100110100011101000101010010000100000011011110110001001101010011001010110001110001001011101101010010110101010101000100111001010110100011100110110110111000100100010010110111001100111001000000110110010101110011001111010010101110110011011110110110001110101011011011001011011001000101010110110100101101110011001100110111101110010011011011000100010001010101010011100111101110010110110111000101101100111000010100110010001101001100100001011010001100011100101100